In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


###Uploading Data

In [42]:
df = pd.read_csv('/content/drive/MyDrive/detroit data/final_cleaned_detroit.csv')
df.head()

,Parcel ID,Taxpayer 1,Taxpayer City,Property Class,Total Floor Area,Total Acreage,Frontage,Depth,Building Count,Year Built,Sale Price,Sale Date,Assessed Value,Previous Assessed Value,Taxable Value,Previous Taxable Value,Neighborhood,fine_amount,yearly_average,vacant,Council District,Binary Tax Status,Binary Blight Violation,Binary Building Permit Status,Sale Date Year,neighborhood_population,num_vacant_neighborhood,normcrime
0,21062470.,853 ASHLAND LLC,1,401.0,2468.0,0.079,30.0,115.0,1.0,1916.0,65000.0,2021-09-14,61300.0,46200.0,61300.0,16361.0,Jefferson Chalmers,250.0,1602.2,0,4.0,1,1,0,2021.0,1695.0,49.0,0.945251
1,21063979.,"CASTLE, ROBERT M",0,401.0,1389.0,0.000,0.0,0.0,1.0,2002.0,13600.0,2014-02-14,40100.0,31700.0,13286.0,12654.0,Morningside,0.0,3541.0,0,4.0,1,1,0,2014.0,3606.0,152.0,0.981974
2,21068429.,CITIMORTGAGE INC,0,401.0,1293.0,0.115,40.0,126.0,1.0,1938.0,79992.0,2023-07-27,40400.0,33400.0,40400.0,15835.0,Moross-Morang,0.0,3200.6,0,4.0,1,1,0,2023.0,2610.0,85.0,1.226284
3,21069819.,"ROBINSON, DENISE",1,401.0,1540.0,0.105,40.0,114.0,1.0,1929.0,8912.0,2010-04-30,39900.0,31300.0,14391.0,13706.0,Morningside,0.0,3541.0,0,4.0,1,1,0,2010.0,3606.0,152.0,0.981974
4,21070146.,"BRADFORD, WILLIAM",1,401.0,920.0,0.107,40.0,116.0,1.0,1938.0,20000.0,1987-09-01,20300.0,16200.0,9396.0,8949.0,Outer Drive-Hayes,0.0,4247.8,0,4.0,1,1,0,1987.0,3234.0,304.0,1.313482


###Extra Cleaning-move to main

In [43]:
df = df.dropna(subset=["Previous Assessed Value"])
df = df.dropna(subset=["neighborhood_population"])
df = df.dropna(subset=["num_vacant_neighborhood"])
df = df.dropna(subset=["normcrime"])

In [44]:
df["fine_amount"] = df["fine_amount"].fillna(0)

###Randomly removing 10% of rows for final test

In [45]:
df_remaining = df

#dont use below for now
#df_10_percent = df.sample(frac=0.1, random_state=42)

#df_remaining = df.drop(df_10_percent.index)

In [46]:
#dont use for now
#X_test_final = df_10_percent[["Total Floor Area", "Total Acreage", "Frontage", "Depth", "Sale Price", "Taxpayer City is Detroit", "Binary Blight Violation", "Binary Building Permit Status", "fine_amount"]]
#y_test_final = df_10_percent["vacant"]

###Variable Selection and Train Test Split

In [47]:
#establishing x and y data
X = df_remaining[["Total Floor Area", "Total Acreage", "Frontage", "Depth", "Sale Price", "Taxpayer City", "Binary Blight Violation", "Binary Building Permit Status", "fine_amount", "neighborhood_population", "num_vacant_neighborhood", "normcrime", "Assessed Value"]] #leaving out "Sale Date Year", "Year Built",
#X = df.drop(columns=["vacant", "Taxpayer 1", "Neighborhood", "Binary Blight Violation", "Parcel ID"])
y = df_remaining["vacant"]

In [48]:
#creeating randomized train/test split with random sectors of 41 (test=20% of data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [49]:
X_train.head(20)

,Total Floor Area,Total Acreage,Frontage,Depth,Sale Price,Taxpayer City,Binary Blight Violation,Binary Building Permit Status,fine_amount,neighborhood_population,num_vacant_neighborhood,normcrime,Assessed Value
52187,699.0,0.100,35.0,124.0,17700.0,1,1,0,0.0,565.0,90.0,1.556106,15200.0
174425,1184.0,0.089,35.0,110.0,10000.0,1,1,0,50.0,1772.0,155.0,1.475169,28500.0
137882,1088.0,0.103,36.0,125.0,33000.0,1,1,0,0.0,3308.0,180.0,1.323579,36000.0
165953,854.0,0.132,40.0,143.0,0.0,1,1,0,0.0,2810.0,102.0,0.793025,14500.0
130878,841.0,0.079,30.0,114.0,0.0,1,1,0,0.0,2980.0,182.0,1.133087,28800.0
516,1109.0,0.110,40.0,120.0,60000.0,0,1,0,0.0,3606.0,152.0,0.981974,29700.0
106445,728.0,0.197,50.0,172.0,15000.0,0,1,0,50.0,852.0,11.0,1.203756,25400.0
150286,1266.0,0.124,45.0,120.0,95650.0,1,1,0,200.0,609.0,14.0,2.086700,45500.0
169951,1124.0,0.081,32.0,110.0,114250.0,0,1,0,0.0,3204.0,30.0,1.169288,21300.0
13917,1919.0,0.138,40.0,150.0,21000.0,1,1,0,0.0,1149.0,105.0,0.868930,29900.0


In [50]:
y_train.head()
y_train.value_counts()

,count
vacant,
0,158920
1,10263


In [51]:
y_test.head()
y_test.value_counts()

,count
vacant,
0,39776
1,2520


###Sequential Deep Neural Network for Binary Classification with Dropout and Weights

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np


y_train_array = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train

#class weights which can be changed
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_array),
                                                  y=y_train_array)

#keras needs it to be a dictionary
class_weights = dict(enumerate(class_weights))
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)



#model parameters
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #this automatically gets features for first layer
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

#compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

#run model
history = model.fit(X_train, y_train_array, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test), class_weight=class_weights, callbacks=[early_stopping])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.7640 - loss: 0.5654 - val_accuracy: 0.9184 - val_loss: 0.3884
Epoch 2/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8706 - loss: 0.4622 - val_accuracy: 0.9120 - val_loss: 0.3851
Epoch 3/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.8738 - loss: 0.4349 - val_accuracy: 0.9008 - val_loss: 0.4100
Epoch 4/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8725 - loss: 0.4389 - val_accuracy: 0.8924 - val_loss: 0.4173
Epoch 5/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8651 - loss: 0.4362 - val_accuracy: 0.9024 - val_loss: 0.4022
Epoch 6/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.8731 - loss: 0.4329 - val_accuracy: 0.9059 - val_loss: 0.3747
Epoch 7/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8708 - loss: 0.4325 - val_accuracy: 0.8957 - val_loss: 0.4004
Epoch 8/50
5287/5287 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.8646 - loss: 0

###Testing metrics: will add more here

In [53]:
len(X_test)

42296

In [54]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

1322/1322 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9063 - loss: 0.3747
Test Accuracy: 0.9058539867401123


###Matrix Results

In [55]:


predictions = model.predict(X_test)
predictions_binary = (predictions > 0.5).astype(int)  # Convert to binary labels

cm = confusion_matrix(y_test, predictions_binary)
print("Confusion Matrix:")
print(cm)

#[[TN, FP],
#[[FN, TP]]

1322/1322 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Confusion Matrix:
[[36496  3280]
 [  702  1818]]


In [56]:
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(f"Incorrectly predicted 'Vacant' (False Positives): {FP}")
print(f"Incorrectly predicted 'Not Vacant' (False Negatives): {FN}")

Incorrectly predicted 'Vacant' (False Positives): 3280
Incorrectly predicted 'Not Vacant' (False Negatives): 702


In [ ]:
#optimizing thresholds and use beeswarm plot